In [1]:

import os as os
import scipy.io as spio
import numpy as np

from fooof import FOOOF, FOOOFGroup
from fooof.plts.annotate import plot_annotated_peak_search

import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import sys


%matplotlib inline

/var/folders/f9/c7c43dks1tq0xh5s764xfnxh0000gn/T/ipykernel_82616/2747575886.py:5: DeprecationWarning: 
The `fooof` package is being deprecated and replaced by the `specparam` (spectral parameterization) package.
This version of `fooof` (1.1) is fully functional, but will not be further updated.
New projects are recommended to update to using `specparam` (see Changelog for details).
  from fooof import FOOOF, FOOOFGroup


In [14]:
data = spio.loadmat('/Users/ecesnaite/Desktop/BuschLab/Ramybe-rsEEG/analysis/pz_psd_groups.mat', squeeze_me=True)#a .mat file structure that contains all subjects (as rows) and their PSD values for each EEG channel in 'psd' structure
data_all = data['psd']

In [15]:
# Set the frequency range upon which to fit FOOOF
freq_range = [3, 40]

# Initialize the results list
results = [] 
result =[]

In [16]:
subj=0
chan=0

In [17]:
data_all.shape


(174,)

In [18]:
# Specify the directory path 
directory = '/Users/ecesnaite/Desktop/BuschLab/Ramybe-rsEEG/analysis/'

In [20]:
data.shape[0]

4

In [22]:
fm = FOOOF(peak_threshold=2.0)
data = data_all['spect'][0]
name = data_all['ID'][0]
freqs = data_all['freq'][0]

# Create empty dictionaries for each parameter
ap_params_dict = {}
# Iterate over channels
for chan in range(data.shape[0]):
    spectrum = data[chan]

    # Fit the model
    fm.fit(freqs, spectrum, freq_range)
        #fm.report()

    # Get the model fit results
    ap_params, peak_params, r_squared, fit_error, gauss_params = fm.get_results()

    # Save each parameter separately for each channel
    ap_params_dict[f'channel_{chan+1}'] = ap_params # Aperiodic Parameters: offset, exponent
        

In [24]:
# Save each parameter dictionary for the subject as MATLAB .mat file
savename = f'group_avg_ap_parameters.mat'
spio.savemat(savename, mdict={'ap_params': ap_params_dict})

In [ ]:
# Iterate over subjects
for subj in range(4):
    fm = FOOOF(peak_threshold=2.0)
    data = data_all['spect'][subj]
    name = data_all['ID'][subj]
    freqs = data_all['freq'][subj]

    # Create empty dictionaries for each parameter
    ap_params_dict = {}
    peak_params_dict = {}
    r_squared_dict = {}
    fit_error_dict = {}
    gauss_params_dict = {}

    # Iterate over channels
    for chan in range(data.shape[0]):
        spectrum = data[chan]

        # Fit the model
        fm.fit(freqs, spectrum, freq_range)
        #fm.report()

        # Get the model fit results
        ap_params, peak_params, r_squared, fit_error, gauss_params = fm.get_results()

        # Save each parameter separately for each channel
        ap_params_dict[f'channel_{chan+1}'] = ap_params # Aperiodic Parameters: offset, exponent
        peak_params_dict[f'channel_{chan+1}'] = peak_params #  center frequency(CF); power (PW); band width (BW)
        r_squared_dict[f'channel_{chan+1}'] = r_squared 
        fit_error_dict[f'channel_{chan+1}'] = fit_error 
        gauss_params_dict[f'channel_{chan+1}'] = gauss_params #gaussian fit. params: CF; PW; BW

In [ ]:
# Save each parameter dictionary for the subject as MATLAB .mat file
    savename = f'{name}_parameters.mat'
    spio.savemat(savename, mdict={'ap_params': ap_params_dict, 
                                  'peak_params': peak_params_dict,
                                  'r_squared': r_squared_dict,
                                  'fit_error': fit_error_dict,
                                  'gauss_params': gauss_params_dict})

In [ ]:
##### to plot parameters for one subject 


#Specify the identification code of the subject you want to extract parameters for
subj_id = 'UID_102'

# Iterate over subjects
for subj in range(data_all.shape[0]):
    fm = FOOOF(peak_threshold=1.0)
    data = data_all['spect'][subj]
    name = data_all['ID'][subj]
    freqs = data_all['freq'][subj]

    if name == subj_id:
        # Iterate over channels
        for chan in range(data.shape[0]):
            spectrum = data[chan]

            # Fit the model
            fm.fit(freqs, spectrum, freq_range)
            fm.report()

In [ ]:
##### to plot parameters for specific subject and channel

# Specify the identification code of the subject you want to extract parameters for
subj_id = 'NCF_207'
# Specify the channel number you want to extract parameters for
channel_num = 27

# Iterate over subjects
for subj in range(data_all.shape[0]):
    fm = FOOOF(peak_threshold=2.0)
    data = data_all['spect'][subj]
    name = data_all['ID'][subj]
    freqs = data_all['freq'][subj]

    if name == subj_id:
        # Iterate over channels
        for chan in range(data.shape[0]):
            if chan == channel_num:
                spectrum = data[chan]

                # Fit the model
                fm.fit(freqs, spectrum, freq_range)
                fm.report()
                
                plot_annotated_peak_search(fm) # illustrate peak search 